# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Publish-Subscribe (Pub-Sub)

**Subtopic** - Implementing a Pub-Sub System for Real-Time Chat Applications

**Use Case** - Implement a simple pub-sub pattern in Python for a real-time chat application without using Flask/Django or any other complicated framework but rather using a package like pypubsub. Ensure it supports both publishing messages to the topic and subscribing to a topic to receive those messages. Additionally, ensure that the subscribers receive the most recently published messages in real time

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

You are tasked with implementing an event-driven publish-subscribe message distribution system for a multi-room chat application. The system should handle message publishing to specific chat rooms and ensure all subscribers of those rooms receive messages in the correct order. The system must support dynamic subscription management, message persistence for late joiners, and handle various edge cases that occur in real-world chat applications.

## Input Format

The input is a list of strings where each string represents an operation:
- SUBSCRIBE <user_id> <room_id> - User subscribes to a room
- UNSUBSCRIBE <user_id> <room_id> - User unsubscribes from a room  
- PUBLISH <room_id> <sender_id> <message> - Publish message to room
- GET_MESSAGES <user_id> <room_id> - Get all messages for user in room
- GET_ACTIVE_USERS <room_id> - Get list of active subscribers in room
- CLEAR_ROOM <room_id> - Clear all messages and subscribers from room

## Output Format

The output is a list of strings where each string corresponds to the result of the operation at the same index in the input list:
- SUBSCRIBE: OK or ALREADY_SUBSCRIBED
- UNSUBSCRIBE: OK or NOT_SUBSCRIBED
- PUBLISH: DELIVERED_TO_<count>_USERS
- GET_MESSAGES: [<sender_id>: <message>, <sender_id>: <message>, ...]
- GET_ACTIVE_USERS: [<user_id>, <user_id>, ...] (sorted alphabetically)
- CLEAR_ROOM: OK

## Examples

### Example 1

Input:

```
[
    "SUBSCRIBE alice room1",
    "SUBSCRIBE bob room1",
    "PUBLISH room1 alice Hello everyone",
    "GET_MESSAGES alice room1",
    "GET_MESSAGES bob room1"
]
```

Output:

```
[
    "OK",
    "OK",
    "DELIVERED_TO_2_USERS",
    "[alice: Hello everyone]",
    "[alice: Hello everyone]"
]
```

### Example 2

Input:

```
[
    "SUBSCRIBE user1 room1",
    "SUBSCRIBE user2 room1",
    "PUBLISH room1 user1 First message",
    "SUBSCRIBE user3 room1",
    "PUBLISH room1 user2 Second message",
    "GET_MESSAGES user3 room1",
    "GET_ACTIVE_USERS room1",
    "UNSUBSCRIBE user1 room1",
    "GET_ACTIVE_USERS room1"
]
```

Output:

```
[
    "OK",
    "OK",
    "DELIVERED_TO_2_USERS",
    "OK",
    "DELIVERED_TO_3_USERS",
    "[user1: First message, user2: Second message]",
    "[user1, user2, user3]",
    "OK",
    "[user2, user3]"
]
```

# Requirements

## Explicit Requirements

1. Users must be subscribed to a room to receive messages from that room
2. Messages published to a room are delivered to ALL current subscribers
3. Users subscribing to a room after messages were published should receive all previous messages
4. Message order must be preserved (first published, first in list)
5. User IDs and room IDs are case-sensitive
6. Empty rooms should return empty lists for messages and active users
7. Publishing to a room with no subscribers should return DELIVERED_TO_0_USERS
8. It should be an event-driven system

## Implicit Requirements

1. System should handle multiple rooms simultaneously without interference
2. Same user can subscribe to multiple rooms
3. Same user cannot subscribe to the same room twice
4. Unsubscribing from a non-subscribed room should return appropriate status
5. Messages persist in rooms even after the original sender unsubscribes
6. System should maintain state between operations
7. Room and user management should be dynamic (created on first use)

## Function Signature

```python
def process_chat_operations(operations):
    """
    Process a list of chat room operations and return results.
    
    Args:
        operations (list): List of operation strings
        
    Returns:
        list: List of result strings corresponding to each operation
    """
```

## Edge Cases

1. When someone asks for messages or active users from a room that doesn't exist, show them an empty list
2. Users can send messages to rooms they are subscribed to
3. If someone tries to join a room they're already in, tell them they're already subscribed
4. If someone tries to leave a room they're not in, tell them they're not subscribed
5. When a message is sent to a room with no subscribers, report that it was delivered to zero users
6. Handle messages that contain multiple words and special characters properly
7. After a room is cleared, treat it as completely new and empty for all future operations
8. Allow anyone to send messages to any room, even if they're not subscribed to it
9. Treat empty strings as valid messages that can be sent and received
10. Treat "Room1" and "room1" as completely different rooms

## Constraints

- User IDs contain only alphanumeric characters and underscores (1 <= length <= 20)
- Room IDs contain only alphanumeric characters and underscores (1 <= length <= 20)  
- Messages can contain any printable characters including spaces (0 <= length <= 200)
- Maximum 1000 operations per test case
- Maximum 100 unique users per room
- Maximum 50 unique rooms
- Maximum 500 messages per room
- Operations are processed sequentially in the order given
- All operations are guaranteed to be in valid format
- No concurrent access considerations needed (single-threaded execution)
- Your response and code must be complete, else it isn't acceptable.
- Your solution should be contained in single file and must have driver function.
- Implementation code must run on an online python compiler and must be testable.

In [ ]:
# code

class ChatEvent:
    """Represents an event in the chat system."""

    def __init__(self, event_type, room_id, user_id=None,
                 message=None, sender_id=None):
        """
        Initialize the event with type, room ID, user ID, message,
        and sender ID.
        """
        self.event_type = event_type
        self.room_id = room_id
        self.user_id = user_id
        self.message = message
        self.sender_id = sender_id


class ChatRoom:
    """Represents a chat room with subscribers and messages."""

    def __init__(self, room_id):
        """
        Initialize the chat room with a room ID,
        an empty set of subscribers, and an empty message.
        """
        self.room_id = room_id
        self.subscribers = set()
        self.messages = []

    def add_subscriber(self, user_id):
        """Add a subscriber to the room."""
        if user_id in self.subscribers:
            return False
        self.subscribers.add(user_id)
        return True

    def remove_subscriber(self, user_id):
        """Remove a subscriber from the room."""
        if user_id not in self.subscribers:
            return False
        self.subscribers.remove(user_id)
        return True

    def add_message(self, sender_id, message):
        """Add a message to the room."""
        self.messages.append((sender_id, message))

    def get_messages(self):
        """Get all messages in the room."""
        return self.messages.copy()

    def get_active_users(self):
        """Get sorted list of active subscribers."""
        return sorted(list(self.subscribers))

    def clear(self):
        """Clear all messages and subscribers."""
        self.subscribers.clear()
        self.messages.clear()


class EventHandler:
    """Handles different types of events."""

    def __init__(self, chat_system):
        """Initialize with a reference to the chat system."""
        self.chat_system = chat_system

    def handle_event(self, event):
        """Route events to appropriate handlers"""
        if event.event_type == "SUBSCRIBE":
            return self.handle_subscribe(event)
        elif event.event_type == "UNSUBSCRIBE":
            return self.handle_unsubscribe(event)
        elif event.event_type == "PUBLISH":
            return self.handle_publish(event)
        elif event.event_type == "GET_MESSAGES":
            return self.handle_get_messages(event)
        elif event.event_type == "GET_ACTIVE_USERS":
            return self.handle_get_active_users(event)
        elif event.event_type == "CLEAR_ROOM":
            return self.handle_clear_room(event)

    def handle_subscribe(self, event):
        """Handle subscription event."""
        room = self.chat_system.get_or_create_room(event.room_id)
        success = room.add_subscriber(event.user_id)
        return "OK" if success else "ALREADY_SUBSCRIBED"

    def handle_unsubscribe(self, event):
        """Handle unsubscription event."""
        room = self.chat_system.get_room(event.room_id)
        if room is None:
            return "NOT_SUBSCRIBED"
        success = room.remove_subscriber(event.user_id)
        return "OK" if success else "NOT_SUBSCRIBED"

    def handle_publish(self, event):
        """Handle message publish event."""
        room = self.chat_system.get_or_create_room(event.room_id)
        room.add_message(event.sender_id, event.message)
        subscriber_count = len(room.subscribers)
        return f"DELIVERED_TO_{subscriber_count}_USERS"

    def handle_get_messages(self, event):
        """Handle get messages event."""
        room = self.chat_system.get_room(event.room_id)
        if room is None:
            return "[]"

        messages = room.get_messages()
        if not messages:
            return "[]"

        message_strings = [f"{sender}: {msg}" for sender, msg in messages]
        return f"[{', '.join(message_strings)}]"

    def handle_get_active_users(self, event):
        """Handle get active users event."""
        room = self.chat_system.get_room(event.room_id)
        if room is None:
            return "[]"

        users = room.get_active_users()
        if not users:
            return "[]"

        return f"[{', '.join(users)}]"

    def handle_clear_room(self, event):
        """Handle clear room event."""
        room = self.chat_system.get_room(event.room_id)
        if room is not None:
            room.clear()
        return "OK"


class ChatSystem:
    """Main chat system that manages rooms and processes events."""

    def __init__(self):
        """
        Initialize the chat system with an empty room dictionary
        and event handler.
        """
        self.rooms = {}
        self.event_handler = EventHandler(self)

    def get_room(self, room_id):
        """Get existing room or None if doesn't exist."""
        return self.rooms.get(room_id)

    def get_or_create_room(self, room_id):
        """Get existing room or create new one."""
        if room_id not in self.rooms:
            self.rooms[room_id] = ChatRoom(room_id)
        return self.rooms[room_id]

    def process_event(self, event):
        """Process a single event and return result."""
        return self.event_handler.handle_event(event)

    def parse_operation(self, operation):
        """Parse operation string into ChatEvent."""
        parts = operation.split(' ', 3)

        if parts[0] == "SUBSCRIBE":
            return ChatEvent("SUBSCRIBE", parts[2], user_id=parts[1])
        elif parts[0] == "UNSUBSCRIBE":
            return ChatEvent("UNSUBSCRIBE", parts[2], user_id=parts[1])
        elif parts[0] == "PUBLISH":
            room_id = parts[1]
            sender_id = parts[2]
            message = parts[3] if len(parts) > 3 else ""
            return ChatEvent(
                "PUBLISH", room_id, sender_id=sender_id, message=message)
        elif parts[0] == "GET_MESSAGES":
            return ChatEvent("GET_MESSAGES", parts[2], user_id=parts[1])
        elif parts[0] == "GET_ACTIVE_USERS":
            return ChatEvent("GET_ACTIVE_USERS", parts[1])
        elif parts[0] == "CLEAR_ROOM":
            return ChatEvent("CLEAR_ROOM", parts[1])


def process_chat_operations(operations):
    """
    Process a list of chat room operations and return results.

    Args:
        operations (list): List of operation strings

    Returns:
        list: List of result strings corresponding to each operation
    """
    chat_system = ChatSystem()
    results = []

    for operation in operations:
        # Parse operation into event
        event = chat_system.parse_operation(operation)

        # Process event and get result
        result = chat_system.process_event(event)
        results.append(result)

    return results


# Driver function with test cases
def main():
    """
    To run test cases for the chat system.
    It processes predefined operations and prints results.
    """
    # Test Case 1
    print("Test Case 1:")
    operations1 = [
        "SUBSCRIBE alice room1",
        "SUBSCRIBE bob room1",
        "PUBLISH room1 alice Hello everyone",
        "GET_MESSAGES alice room1",
        "GET_MESSAGES bob room1"
    ]

    results1 = process_chat_operations(operations1)
    print(results1)

    # Test Case 2
    print("Test Case 2:")
    operations2 = [
        "SUBSCRIBE user1 room1",
        "SUBSCRIBE user2 room1",
        "PUBLISH room1 user1 First message",
        "SUBSCRIBE user3 room1",
        "PUBLISH room1 user2 Second message",
        "GET_MESSAGES user3 room1",
        "GET_ACTIVE_USERS room1",
        "UNSUBSCRIBE user1 room1",
        "GET_ACTIVE_USERS room1"
    ]

    results2 = process_chat_operations(operations2)
    print(results2)

    # Test Case 3 - Edge Cases
    print("Test Case 3 - Edge Cases:")
    operations3 = [
        "GET_MESSAGES user1 nonexistent",
        "GET_ACTIVE_USERS nonexistent",
        "SUBSCRIBE user1 room1",
        "SUBSCRIBE user1 room1",
        "UNSUBSCRIBE user2 room1",
        "PUBLISH room1 user1 ",
        "CLEAR_ROOM room1",
        "GET_MESSAGES user1 room1"
    ]

    results3 = process_chat_operations(operations3)
    print(results3)


if __name__ == "__main__":
    main()


In [ ]:
# tests
"""Unit tests for the ChatSystem module."""

import unittest
from main import ChatSystem, ChatEvent


class TestChatSystem(unittest.TestCase):
    """Test cases for the ChatSystem class."""

    def setUp(self):
        """Set up a fresh ChatSystem instance before each test."""
        self.chat = ChatSystem()

    def test_subscribe_success(self):
        """Test subscribing a user to a room."""
        event = ChatEvent("SUBSCRIBE", "room1", user_id="alice")
        result = self.chat.process_event(event)
        self.assertEqual(result, "OK")

    def test_subscribe_duplicate(self):
        """Test subscribing the same user twice returns ALREADY_SUBSCRIBED."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="alice")
        )
        result = self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="alice")
        )
        self.assertEqual(result, "ALREADY_SUBSCRIBED")

    def test_unsubscribe_success(self):
        """Test unsubscribing a user who is subscribed."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="alice")
        )
        result = self.chat.process_event(
            ChatEvent("UNSUBSCRIBE", "room1", user_id="alice")
        )
        self.assertEqual(result, "OK")

    def test_unsubscribe_not_subscribed(self):
        """Test unsubscribing a user who is not subscribed."""
        result = self.chat.process_event(
            ChatEvent("UNSUBSCRIBE", "room1", user_id="bob")
        )
        self.assertEqual(result, "NOT_SUBSCRIBED")

    def test_publish_to_empty_room(self):
        """Test publishing a message to a room with no subscribers."""
        result = self.chat.process_event(
            ChatEvent("PUBLISH", "room1", sender_id="alice", message="Hi")
        )
        self.assertEqual(result, "DELIVERED_TO_0_USERS")

    def test_publish_to_subscribers(self):
        """Test publishing a message to subscribed users."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="a")
        )
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="b")
        )
        result = self.chat.process_event(
            ChatEvent("PUBLISH", "room1", sender_id="a", message="msg")
        )
        self.assertEqual(result, "DELIVERED_TO_2_USERS")

    def test_get_messages(self):
        """Test retrieving messages from a room."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="a")
        )
        self.chat.process_event(
            ChatEvent("PUBLISH", "room1", sender_id="a", message="hello")
        )
        result = self.chat.process_event(
            ChatEvent("GET_MESSAGES", "room1", user_id="a")
        )
        self.assertEqual(result, "[a: hello]")

    def test_get_messages_empty(self):
        """Test retrieving messages from an empty room."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="a")
        )
        result = self.chat.process_event(
            ChatEvent("GET_MESSAGES", "room1", user_id="a")
        )
        self.assertEqual(result, "[]")

    def test_get_messages_room_not_exist(self):
        """Test retrieving messages from a non-existent room."""
        result = self.chat.process_event(
            ChatEvent("GET_MESSAGES", "nope", user_id="x")
        )
        self.assertEqual(result, "[]")

    def test_get_active_users(self):
        """Test retrieving active users from a room."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="b")
        )
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="a")
        )
        result = self.chat.process_event(
            ChatEvent("GET_ACTIVE_USERS", "room1")
        )
        self.assertEqual(result, "[a, b]")

    def test_get_active_users_empty(self):
        """Test retrieving active users from an empty room."""
        self.chat.get_or_create_room("room1")
        result = self.chat.process_event(
            ChatEvent("GET_ACTIVE_USERS", "room1")
        )
        self.assertEqual(result, "[]")

    def test_get_active_users_room_not_exist(self):
        """Test retrieving users from a non-existent room."""
        result = self.chat.process_event(
            ChatEvent("GET_ACTIVE_USERS", "nope")
        )
        self.assertEqual(result, "[]")

    def test_clear_room(self):
        """Test clearing a room with subscribers and messages."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="a")
        )
        self.chat.process_event(
            ChatEvent("PUBLISH", "room1", sender_id="a", message="x")
        )
        self.chat.process_event(ChatEvent("CLEAR_ROOM", "room1"))
        result = self.chat.process_event(
            ChatEvent("GET_MESSAGES", "room1", user_id="a")
        )
        self.assertEqual(result, "[]")

    def test_clear_room_not_exist(self):
        """Test clearing a room that does not exist."""
        result = self.chat.process_event(
            ChatEvent("CLEAR_ROOM", "ghost")
        )
        self.assertEqual(result, "OK")

    def test_publish_empty_message(self):
        """Test publishing an empty message."""
        self.chat.process_event(
            ChatEvent("SUBSCRIBE", "room1", user_id="u")
        )
        result = self.chat.process_event(
            ChatEvent("PUBLISH", "room1", sender_id="u", message="")
        )
        self.assertEqual(result, "DELIVERED_TO_1_USERS")

    def test_parse_operation_subscribe(self):
        """Test parsing a SUBSCRIBE operation string."""
        op = "SUBSCRIBE alice room42"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.event_type, "SUBSCRIBE")
        self.assertEqual(event.user_id, "alice")
        self.assertEqual(event.room_id, "room42")

    def test_parse_operation_publish(self):
        """Test parsing a PUBLISH operation with a message."""
        op = "PUBLISH room42 userX Hello"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.event_type, "PUBLISH")
        self.assertEqual(event.room_id, "room42")
        self.assertEqual(event.sender_id, "userX")
        self.assertEqual(event.message, "Hello")

    def test_parse_operation_publish_no_message(self):
        """Test parsing a PUBLISH operation with no message."""
        op = "PUBLISH room42 userX"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.message, "")

    def test_parse_operation_get_messages(self):
        """Test parsing a GET_MESSAGES operation string."""
        op = "GET_MESSAGES bob roomX"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.event_type, "GET_MESSAGES")
        self.assertEqual(event.user_id, "bob")
        self.assertEqual(event.room_id, "roomX")

    def test_parse_operation_get_active_users(self):
        """Test parsing a GET_ACTIVE_USERS operation string."""
        op = "GET_ACTIVE_USERS roomY"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.event_type, "GET_ACTIVE_USERS")
        self.assertEqual(event.room_id, "roomY")

    def test_parse_operation_clear_room(self):
        """Test parsing a CLEAR_ROOM operation string."""
        op = "CLEAR_ROOM testRoom"
        event = self.chat.parse_operation(op)
        self.assertEqual(event.event_type, "CLEAR_ROOM")
        self.assertEqual(event.room_id, "testRoom")


# Model Breaking Proof

#### Model Breaking Task URL: https://labeling-ms.turing.com/conversations/68001/view

#### Model code:

```python
def process_chat_operations(operations):
    # <Issue>: Model's implementation is not event-driven and lacks publisher
    # /subscriber abstraction; instead uses procedural approach
    room_subscribers = {}  # {"room_id": set("user1", "user2", ...)}
    room_messages = {}     # {"room_id": [("sender_id", "message"), ...]}
    # <Issue>: Model violates requirement by tracking last read indexes;
    #  instead all subscribers should get ALL messages
    user_last_message_index = {}  # {"user_id": {"room_id": last_index}}

    results = []

    for operation in operations:
        # <Issue>: Parsing bug, resulting in IndexError for PUBLISH commands
        parts = operation.split(' ', 2)
        command = parts[0]

        if command == "SUBSCRIBE":
            user_id, room_id = parts[1], parts[2]

            # Initialize room data structures if they don't exist
            if room_id not in room_subscribers:
                room_subscribers[room_id] = set()
                room_messages[room_id] = []

            if room_id not in user_last_message_index.get(user_id, {}):
                user_last_message_index.setdefault(user_id, {})
                user_last_message_index[user_id][room_id] = 0
            
            if user_id in room_subscribers[room_id]:
                results.append("ALREADY_SUBSCRIBED")
            else:
                room_subscribers[room_id].add(user_id)
                results.append("OK")

        elif command == "UNSUBSCRIBE":
            user_id, room_id = parts[1], parts[2]

            if room_id in room_subscribers and user_id in room_subscribers[room_id]:
                room_subscribers[room_id].remove(user_id)
                results.append("OK")
            else:
                results.append("NOT_SUBSCRIBED")

        elif command == "PUBLISH":
            room_id, sender_id, message = parts[1], parts[2], parts[3]

            if room_id not in room_messages:
                room_messages[room_id] = []

            # Append message to the room's message list
            room_messages[room_id].append((sender_id, message))

            # Deliver the message to all subscribers
            if room_id not in room_subscribers:
                results.append("DELIVERED_TO_0_USERS")
            else:
                num_users = len(room_subscribers[room_id])
                results.append(f"DELIVERED_TO_{num_users}_USERS")

        elif command == "GET_MESSAGES":
            user_id, room_id = parts[1], parts[2]

            if room_id not in room_messages:
                results.append("[]")
            else:
                last_index = user_last_message_index.get(user_id, {}).get(room_id, 0)
                message_list = room_messages[room_id][last_index:]
                user_last_message_index[user_id][room_id] = len(room_messages[room_id])
                # <Issue>: Model violates explicit requirement;
                # users should get ALL previous messages, not just new ones

                if message_list:
                    formatted_messages = [f"{sender_id}: {message}" for sender_id, message in message_list]
                    results.append(f"[{', '.join(formatted_messages)}]")
                else:
                    results.append("[]")

        elif command == "GET_ACTIVE_USERS":
            room_id = parts[1]
            if room_id not in room_subscribers or not room_subscribers[room_id]:
                results.append("[]")
            else:
                active_users = sorted(room_subscribers[room_id])
                results.append(f"[{', '.join(active_users)}]")

        elif command == "CLEAR_ROOM":
            room_id = parts[1]
            if room_id in room_subscribers:
                room_subscribers[room_id].clear()

            if room_id in room_messages:
                room_messages[room_id] = []
            # <Issue>: Doesn't clear user_last_message_index for the room

            results.append("OK")

    return results

# Example usage:
operations1 = [
    "SUBSCRIBE alice room1",
    "SUBSCRIBE bob room1",
    "PUBLISH room1 alice Hello everyone",
    "GET_MESSAGES alice room1",
    "GET_MESSAGES bob room1"
]

operations2 = [
    "SUBSCRIBE user1 room1",
    "SUBSCRIBE user2 room1",
    "PUBLISH room1 user1 First message",
    "SUBSCRIBE user3 room1",
    "PUBLISH room1 user2 Second message",
    "GET_MESSAGES user3 room1",
    "GET_ACTIVE_USERS room1",
    "UNSUBSCRIBE user1 room1",
    "GET_ACTIVE_USERS room1"
]

print(process_chat_operations(operations1))
print(process_chat_operations(operations2))
```